# Brown Dwarf Models

Modeling brown dwarfs is very similar to modeling thermal emission for exoplanets. The major difference is there is no stellar input to get out a relative flux (fp/f*) 

In this tutorial you will learn:

1. How to turn that feature off 
2. Query a profile from the Sonora Grid (not you will need to download and untar this to provide a filepath.) 
3. Create a Brown Dwarf Spectrum 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import astropy.units as u

#picaso
import picaso.opacity_factory as opa_fa
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
from scipy.stats.stats import pearsonr  

output_notebook()
from bokeh.plotting import show,figure

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
Start with the same inputs as before

In [7]:
wave_range = [1,1.4]
#filename_db = '/Users/nbatalh1/Documents/data/all_opacities_noregrid.db'
filename_db = 'opacities.db'
opa = jdi.opannection(wave_range=wave_range
                      ,filename_db=filename_db)

case1 = jdi.inputs(calculation='browndwarf')



NameError: name 'jdi' is not defined

In [8]:
# Note here that we do not need to provide case.star, since there is none! 
case1.gravity(gravity=100 , gravity_unit=u.Unit('m/s**2'))


NameError: name 'case1' is not defined

## Query from Sonora Profile Grid 

Download the profile files that are located in the [profile.tar file](https://zenodo.org/record/1309035#.Xo5GbZNKjGJ)

Once you untar the file you can set the file path below. You do not need to unzip each profile. `picaso` will do that upon read in. `picaso` will find the nearest neighbor and attach it to your class. 

In [9]:
#grab sonora profile 
sonora_profile_db = '/Users/nbatalh1/Documents/data/sonora_profile/'
Teff = 900 
#this function will grab the gravity you have input above and find the nearest neighbor with the 
#note sonora chemistry grid is on the same opacity grid as our opacities (1060). 
case1.sonora(sonora_profile_db, Teff)

NameError: name 'case1' is not defined

In [10]:
#check what file was chosen
print(case1.inputs['atmosphere']['sonora_filename'])

NameError: name 'case1' is not defined

In [ ]:
#look at your input
case1.inputs['atmosphere']['profile'].head()

## Run Spectrum

In [ ]:
df = case1.spectrum(opa, full_output=True)

## Convert Units and Regrid

`PICASO` outputs the raw flux as: 

$$ F_\lambda ( \frac{erc}{cm^2 * sec * cm}) $$

Typical fluxes shown in several Brown Dwarf papers are: 

$$ F_\nu ( \frac{erc}{cm^2 * sec * Hz}) $$

Below is a little example of how to convert units. 

**NOTE**: Some people like to plot out `Eddington Flux`, $H_\nu$. This gets confusing as the units appear to be erg/cm2/s/Hz but you will notice a factor of four difference: 

$$H_\nu = \frac{F_\nu}{4}$$

In [11]:
x,y = df['wavenumber'], df['thermal'] #units of erg/cm2/s/cm

xmicron = 1e4/x

flamy = y*1e-8 #per anstrom instead of per cm
sp = jdi.psyn.ArraySpectrum(xmicron, flamy, 
                            waveunits='um', 
                            fluxunits='FLAM')   
sp.convert("um")
sp.convert('Fnu') #erg/cm2/s/Hz

x = sp.wave #micron
y= sp.flux #erg/cm2/s/Hz

x,y = jdi.mean_regrid(1e4/x, y, min_R=1000) #wavenumber, erg/cm2/s/Hz


NameError: name 'df' is not defined

## Compare with Sonora Grid

In [12]:
son = pd.read_csv('sp_t900g100nc_m0.0',delim_whitespace=True, 
                 skiprows=3,header=None,names=['w','f'])
#sonx, sony =  jdi.mean_regrid(1e4/son['w'][::-1], 
#                              son['f'][::-1], newx=x)

FileNotFoundError: [Errno 2] File sp_t900g100nc_m0.0 does not exist: 'sp_t900g100nc_m0.0'

In [ ]:
show(jpi.spectrum([1e4/son['w'],x],[son['f'],y], legend=['picaso','sonora']
                  ,plot_width=800,x_range=wave_range))#x_axis_type='log',

